In [1]:
import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from linear_probe import get_linear_acc
import random
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve

In [2]:
data_reader=ReadData('/root/autodl-tmp/hallscope/ChartVQA/train')
data=data_reader.read_all()
random.shuffle(data)

In [3]:
df=pd.DataFrame(data)

In [ ]:
df.head()

,img_path,question,answers,question_id,most_likely,responses
0,[/root/autodl-tmp/ChartVQA/train/png/8031.png],[What does orange bar represents?\nAnswer the ...,[Better],[8031.png],{'embedding': [[[ 0.00099182 0.0020752 -0.00...,None
1,[/root/autodl-tmp/ChartVQA/train/png/179272210...,[What is the difference of highest value and t...,[0.22],[17927221005995.png],{'embedding': [[[-8.1634521e-04 -1.1672974e-03...,None
2,[/root/autodl-tmp/ChartVQA/train/png/17770.png],[What's the average of all the Oppose data poi...,[32],[17770.png],{'embedding': [[[-0.00982666 0.00026894 0.00...,None
3,[/root/autodl-tmp/ChartVQA/train/png/11017.png],[What's the total percentage of people choosin...,[38],[11017.png],{'embedding': [[[-0.00159454 0.00074005 -0.00...,None
4,[/root/autodl-tmp/ChartVQA/train/png/784461810...,[What does the Green bar represent?\nAnswer th...,[Southern Asia],[78446181005186.png],{'embedding': [[[-0.00946045 -0.01708984 -0.00...,None


In [5]:
i=111
df.iloc[i]['most_likely']['response']

['Egypt']

In [6]:
df.iloc[i]['question']

['Which country is higher in 2018?\nAnswer the question using a single word or digit number.\n']

In [7]:
df.iloc[i]['answers']

['Syria']

In [8]:
embeddings=df['most_likely'].apply(lambda x: x['embedding']).to_list()

In [9]:
embeddings=np.concatenate(embeddings,axis=0)

In [10]:
embeddings.shape

(2226, 29, 3584)

In [11]:
gt_label=df['answers'].to_list()
gt_label=np.concatenate(gt_label,axis=0)

In [12]:
gt_label.shape

(2226,)

In [13]:
responses=df['most_likely'].apply(lambda x: x['response']).to_list()
resps=[]
for x in responses:
    resps+=x

In [14]:
from answer_judge import load_judge

In [15]:
is_correct=[]
judge=load_judge('ChartQA')
for i,x in enumerate(resps):
    if judge.check(x,gt_label[i]):
        is_correct.append(1)
    else:
        is_correct.append(0)
is_correct=np.array(is_correct)

In [16]:
is_correct.shape

(2226,)

In [17]:
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly')

There are 2226 responses. 1628 answer correctly and 598 answer wrongly


In [18]:
length=len(is_correct)
train_rate=0.8

all_indices = np.random.permutation(length)
train_val_idxs = all_indices[:int(
    train_rate * length)]  # trainset and validation index

test_idxs=all_indices[int(
    train_rate * length):]  #test index

validset_len = 1000
# exclude validation samples.
train_idxs = train_val_idxs[:len(
    train_val_idxs) - validset_len]  # trainset index
val_idxs = train_val_idxs[len(
    train_val_idxs) - validset_len:]  # validation index
hall_label_test = []
hall_label_wild = []
hall_label_val = []

'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
hall_label_test = is_correct[test_idxs]
hall_label_wild = is_correct[train_idxs]
hall_label_val = is_correct[val_idxs]

print(f'trainset length:{len(hall_label_wild)}')
print(f'validset length:{len(hall_label_val)}')
print(f'testset length:{len(hall_label_test)}')
print(f'trainset postive: {sum(hall_label_wild==1)} negative: {sum(hall_label_wild==0)}')
print(f'valset postive: {sum(hall_label_val==1)} negative: {sum(hall_label_val==0)}')
print(
    f'testset postive: {sum(hall_label_test==1)} negative: {sum(hall_label_test==0)}')

trainset length:780
validset length:1000
testset length:446
trainset postive: 567 negative: 213
valset postive: 730 negative: 270
testset postive: 331 negative: 115


In [19]:
all_embeddings = embeddings
all_embeddings.shape

(2226, 29, 3584)

In [20]:
embedding_train = all_embeddings[train_idxs]
embedding_val = all_embeddings[val_idxs]
embedding_test = all_embeddings[test_idxs]

In [21]:
%load_ext autoreload
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCADiscriminator
from ml_utils.metrics import auroc,get_best_split_from_scores

Grid Search for Best 'n_components' & 'layer'

In [22]:
def evaluator(n_components,i_layer,X,y):
    kpca=PCADiscriminator(embedding_train[:,i_layer,:],n_components,kernel='linear')
    scores=kpca.get_score2(X[:,i_layer,:])
    return auroc(scores,y)

In [23]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,3),
    'layer':range(embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,hall_label_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

found best n_components:1,layer:0 Score: 0.5247361745306951
found best n_components:1,layer:1 Score: 0.5278792491121258


found best n_components:1,layer:2 Score: 0.5352054794520549
found best n_components:1,layer:3 Score: 0.5450380517503806
found best n_components:1,layer:4 Score: 0.5695636732623033
found best n_components:1,layer:5 Score: 0.586128868594622
found best n_components:1,layer:10 Score: 0.5880162354134957
found best n_components:1,layer:11 Score: 0.5965093860984272


...

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

In [24]:
best_paras

{'n_components': 2, 'layer': 26, 'best_score': np.float64(0.666986301369863)}

In [25]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [26]:
discriminator=PCADiscriminator(embedding_train[:,best_layer,:],best_paras['n_components'],kernel='linear')
scores=discriminator.get_score2(embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(scores,hall_label_val)

In [27]:
scores=discriminator.get_score2(embedding_train[:,best_layer,:])
y_train=(scores>best_split)
print(f'train set acc: {accuracy_score(hall_label_wild,y_train)} AUROC:{roc_auc_score(hall_label_wild,scores)}')

train set acc: 0.5666666666666667 AUROC:0.6280978049366156


In [28]:
scores=discriminator.get_score2(embedding_test[:,best_layer,:])
y_test=(scores>best_split)
print(f'test set acc: {accuracy_score(hall_label_test,y_test)} AUROC:{roc_auc_score(hall_label_test,scores)}')

test set acc: 0.57847533632287 AUROC:0.6631551293839484


Train Linear Prob to Get Better Results

In [29]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=hall_label_test

In [30]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [31]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} AUROC:{roc_auc_score(y_test,scores_test)}')

test set acc: 0.5762331838565022 AUROC:0.6481544726126363
